<a href="https://colab.research.google.com/github/baterdene97/TFG/blob/master/mask_RCNN_cell_nucleus_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Uninstall Tensorflow(once) because by default it doesnt support next codes**

In [ ]:
pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 5.5 MB/s 
     |████████████████████████████████| 15.7 MB 180 kB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatible.
tensorflow 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
google-co

In [ ]:
!pip uninstall tensorflow -y

**Tensorflow version we need:**

In [ ]:
!pip install tensorflow-gpu==1.13.1

**Info of the GPU**

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.')
else:
  print('You have GPU access')
  !nvidia-smi

#print the tensorflow version
print('Tensorflow version is ' + str(tf.__version__))

You have GPU access
Tue Aug 31 15:45:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    58W / 149W |     60MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                    

In [ ]:
!git clone https://github.com/esgomezm/Usiigaci.git

Cloning into 'Usiigaci'...
remote: Enumerating objects: 1479, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 1479 (delta 43), reused 19 (delta 7), pack-reused 1403
Receiving objects: 100% (1479/1479), 470.50 MiB | 18.73 MiB/s, done.
Resolving deltas: 100% (321/321), done.
Checking out files: 100% (429/429), done.


Installing all required dependencies

In [ ]:
import os
os.chdir('/content/Usiigaci/Mask R-CNN/')
!pip3 install -r 'requirements.txt'
!python3 'setup.py' install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding licens

# Mask R-CNN - Train cell nucleus Dataset

Train Mask R-CNN.



In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize

#downloading dataset from github 
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c

!mkdir stage1_train stage1_test

!unzip stage1_train.zip -d stage1_train/
!unzip stage1_test.zip -d stage1_test/

# Data Path
TRAIN_PATH = 'stage1_train/'
TEST_PATH = 'stage1_test/'

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

#print("Please upload the config.py to your google colab")
#from google.colab import files
#src = list(files.upload().values())[0]
#open('config.py','wb').write(src)
#import config

#print("Please upload the utils.py to your google colab")
#from google.colab import files
#src = list(files.upload().values())[0]
#open('utils.py','wb').write(src)
#import utils

#print("Please upload the model.py to your google colab")
#from google.colab import files
#src = list(files.upload().values())[0]
#open('model.py','wb').write(src)
#import model

#print("Please upload the visualize.py to your google colab")
#from google.colab import files
#src = list(files.upload().values())[0]
#open('visualize.py','wb').write(src)
#import visualize

from mrcnn.config import Config
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 
#########################shape=(3,)

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

--2021-08-31 15:45:37--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2021-08-31 15:45:38--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

mkdir: cannot create direct

## Configurations

**Media de celulas**

In [ ]:
import os
suma = 0
total = 0
totalVali = 0
for x in os.listdir(TRAIN_PATH):
  path = TRAIN_PATH + x + '/masks/'
  suma += len(os.listdir(path))
  total += 1
print("Media de masks: ", suma/total)
print("Total de imagenes de train: ", total)

In [ ]:
class ShapesConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides values specific
    to the dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 1 images per GPU. We can put multiple images on each
    # GPU. Batch size is (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 2  # background + nucleus

    # Backbone network architecture: resnet50, resnet101
    BACKBONE = "resnet50"
    # BACKBONE = "resnet101"

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    #IMAGE_MIN_SCALE = 2.0

    IMAGE_RESIZE_MODE = "crop"

    # Use smaller anchors because our image and objects are small
    #RPN_ANCHOR_SCALES = (8, 16, 32, 64,128)  # anchor side in pixels
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256 ) 
    #RPN_ANCHOR_SCALES = (16, 32, 64, 128, 80) #
    #RPN_ANCHOR_SCALES = ( 16, 32, 50, 64, 128)

    RPN_NMS_THRESHOLD = 0.6

    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 500

    RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    # RPN_TRAIN_ANCHORS_PER_IMAGE = 84
    # RPN_TRAIN_ANCHORS_PER_IMAGE = 100

    LEARNING_RATE = 0.001

    MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    #Dont exclude based on confidence
    DETECTION_MIN_CONFIDENCE = 0

    USE_MINI_MASK = True
    #MINI_MASK_SHAPE = (75, 75)      
    MINI_MASK_SHAPE = (56, 56)

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    #TRAIN_ROIS_PER_IMAGE = 90
    TRAIN_ROIS_PER_IMAGE = 128

    # set number of epoch
    STEPS_PER_EPOCH = 45  #

    # set validation steps 
    VALIDATION_STEPS = 5


    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 200
    
config = ShapesConfig()
config.display()

## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [ ]:
class ShapesDataset(utils.Dataset):
    
    def load_shapes(self, mode):
        
        # Add classes
        self.add_class("shapes", 1, "nucleus")
        

        if mode == "train":  
            for n, id_ in enumerate(train_ids):
                if n < int(len(train_ids) * 0.9):
                    path = TRAIN_PATH + id_
                    img_path = path + '/images/'
                    self.add_image("shapes", image_id=id_, path=img_path)
              
        if mode == "val":   
            for n, id_ in enumerate(train_ids):
                if n >= int(len(train_ids) * 0.9):
                    path = TRAIN_PATH + id_
                    img_path = path + '/images/'
                    self.add_image("shapes", image_id=id_, path=img_path)      

    def load_image(self, image_id):
        
        info = self.image_info[image_id]
        info = info.get("id")
       
        path = TRAIN_PATH + info
        img = imread(path + '/images/' + info + '.png')[:,:,:3]
        img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
       
        return img

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        
        info = self.image_info[image_id]
        info = info.get("id")
        path = TRAIN_PATH + info
        number_of_masks = len(next(os.walk(path + '/masks/'))[2])
        mask = np.zeros([config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], number_of_masks], dtype=np.uint8)
        iterator = 0
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = resize(mask_, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
            mask[:, :, iterator] = mask_
            iterator += 1
            # Handle occlusions
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(number_of_masks-2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
            
        # Map class names to class IDs.
        class_ids = np.ones((number_of_masks,), dtype=int)
        
        return mask, class_ids.astype(np.int32)
         
       
        

In [ ]:
# Training dataset
dataset_train = ShapesDataset()
dataset_train.load_shapes("train")
dataset_train.prepare()

# Validation dataset
dataset_val = ShapesDataset()
dataset_val.load_shapes("val")
dataset_val.prepare()

In [ ]:
import os
totalTrain = 0
totalVali = 0
for x in os.listdir('train/'):
  totalTrain += 1
print("Total de imagenes de train: ", totalTrain)

for x in os.listdir('val/'):
  totalVali += 1
print("Total de imagenes de vali: ", totalVali)

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Ceate Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE ,
            epochs=2, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes3.h5")
model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 30 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 30)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
APs = []
APs.append(precisions)
print("Precisions: ",np.mean(APs))
APs = []
APs.append(recalls)
print("recall: ", np.mean(APs))

In [ ]:

X_test = np.zeros((len(test_ids), config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], 3), dtype=np.uint8)
sizes_test = []
_test_ids = []

print('Getting and resizing test images ... ')
#sys.stdout.flush()
for n, id_ in enumerate(test_ids):
    _test_ids.append([id_])
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
    X_test[n] = img

In [ ]:
print("checking a test image with masks ...")
i = 1
results = model.detect([X_test[i]], verbose=1)

r = results[0]
visualize.display_instances(X_test[i], r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())


In [ ]:

model = model.keras_model

## Save model to import it later in DeepImageJ
Now, we will see how to saved the train model into a file so we can later reuse it in the **DeepImageJ** plugin

In [ ]:
import tensorflow as tf
import keras.backend as K
#If the model has only one input it can be converted
OUTPUT_DIR = "DeepImageJ-Mask-RCNN-model.zip"
builder = tf.saved_model.builder.SavedModelBuilder(OUTPUT_DIR)

signature = tf.saved_model.signature_def_utils.predict_signature_def(
            inputs  = {'input_image': 
                       model.inputs[0], 'input_image_meta': model.inputs[1], 'input_anchors': model.inputs[2]},
            outputs = {'mrcnn_detection': model.outputs[0],
                       'mrcnn_class': model.outputs[1],
                       'mrcnn_bbox': model.outputs[2],
                       'mrcnn_mask': model.outputs[3],
                       'ROI': model.outputs[4],
                       'rpn_class': model.outputs[5],
                       'rpn_bbox': model.outputs[6]})
signature_def_map = { tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature }

builder.add_meta_graph_and_variables(K.get_session(), [tf.saved_model.tag_constants.SERVING],
                                             signature_def_map=signature_def_map)
builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: DeepImageJ-Mask-RCNN-model.zip/saved_model.pb


b'DeepImageJ-Mask-RCNN-model.zip/saved_model.pb'

In [ ]:
model.output

[<tf.Tensor 'mrcnn_detection/Reshape_1:0' shape=(1, 200, 6) dtype=float32>,
 <tf.Tensor 'mrcnn_class_1/Reshape_1:0' shape=(?, 1000, 2) dtype=float32>,
 <tf.Tensor 'mrcnn_bbox_1/Reshape:0' shape=(?, 1000, 2, 4) dtype=float32>,
 <tf.Tensor 'mrcnn_mask_1/Reshape_1:0' shape=(?, 200, 28, 28, 2) dtype=float32>,
 <tf.Tensor 'ROI_1/packed_2:0' shape=(1, ?, 4) dtype=float32>,
 <tf.Tensor 'rpn_class_1/concat:0' shape=(?, ?, 2) dtype=float32>,
 <tf.Tensor 'rpn_bbox_1/concat:0' shape=(?, ?, 4) dtype=float32>]

Zip folder and download

In [ ]:
!zip DeepImageJ-Mask-RCNN-model -r DeepImageJ-Mask-RCNN-model

from google.colab import files
files.download("DeepImageJ-Mask-RCNN-model.zip")

	zip warning: missing end signature--probably not a zip file (did you
	zip warning: remember to use binary mode when you transferred it?)
	zip warning: (if you are trying to read a damaged archive try -F)

zip error: Zip file structure invalid (DeepImageJ-Mask-RCNN-model.zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Appendix: calculate the size of the halo ("padding" value in DeepImageJ).

The halo is determined by the receptive field of one pixel in the image: if there was any kind of padding in the convolutions, then the result for the pixels in the corners is affected by this padding. Thus, we need to determine how far from the edges we need to be, in order to avoid the effect of that padding. This is the same as calculating the receptive field of one pixel to determine the cummulative padding performed along the network architecture. 

Therefore, the halo is exactly the size of this receptive field:

*   Two images that are almost the same are analyzed. The first one is a null image and the second one is a null image with a centered unitary pixel. 

*    When processing both images with the model, it seems that the output is the same. However, when the absolute difference is computed, we can notice that it is only the same in the halo, where the convolutions are influenced by the padding and not the content of the image.

In [ ]:
null_im = np.zeros((1,256,256,1))
point_im = np.zeros_like(null_im)
point_im[0,128,128,0] = 1

result_unit = model.predict(np.concatenate((null_im, point_im)), batch_size=1)

plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
plt.imshow(null_im[0,:,:,0])
plt.title('Null image')

plt.subplot(2,2,2)
plt.imshow(point_im[0,:,:,0])
plt.title('Null image with a unitary pixel in the center of the image')

plt.subplot(2,2,3)
plt.imshow(result_unit[0,:,:,0])
plt.colorbar()
plt.title('Prediction of the null image')

plt.subplot(2,2,4)
plt.imshow(result_unit[1,:,:,0])
plt.colorbar()
plt.title('Prediction of unitary point')
plt.show()

In [ ]:
plt.figure()
D = np.abs(result_unit[0]-result_unit[1])>0
D = D[:,:,0]
plt.imshow(D)
plt.title('Absolute differences between both predictions')
plt.show()

In [ ]:
ind = np.where(D[:128,:128]==1)
halo = np.min(ind[1])
print('The size of the halo in this case is {0}'.format(128-halo+1))